<a href="https://colab.research.google.com/github/jixuanlu/MAT494/blob/main/3.7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3.7 Neural Networks TensorFlow 2**

Here we provide a simple example of Neural Network using TensorFlow 2 with MNIST dataset.

Source:

https://www.tensorflow.org/tutorials/quickstart/beginner

In [1]:
import tensorflow as tf

# **1. Load and prepare the MNIST dataset**


1. Load and prepare the MNIST dataset¶
Reference:

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is considered another classic dataset in machine learning area. The images are handwritings of English letters. Researchers are using it as a training set to build neural network models that would recognize human handwritings.

The dataset we import from tf.keras is not the original image set, but numerical matrix that transfromed from the images via certain kernels.

tf.keras.datasets.mnist.load_data() will return tuples of arrays:

- x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.
- y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.
- x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.
- y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.

In [2]:
# load the dataset
mnist = tf.keras.datasets.mnist

# convert the samples from integers to floating-point numbers
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


# **2. Build the tf.keras.Sequential model by stacking layers.**

Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [4]:

predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.05276425,  0.678693  , -0.5726038 ,  0.72082776,  0.5335073 ,
         0.04572374, -0.14213763, -0.33354405,  0.14272729,  0.01601358]],
      dtype=float32)


The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.08675819, 0.16223626, 0.04642123, 0.1692181 , 0.14031191,
        0.08614952, 0.07139465, 0.05895748, 0.09492508, 0.08362765]],
      dtype=float32)


Note: It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [7]:

loss_fn(y_train[:1], predictions).numpy()

2.451671

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2951 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1461 - accuracy: 0.9560
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1083 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0900 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0757 - accuracy: 0.9762


# **3. Evaluate the Model**

The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0748 - accuracy: 0.9764


[0.07479475438594818, 0.9764000177383423]


The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.54472755e-08, 4.84353313e-09, 8.71225097e-07, 1.51904533e-04,
        6.80037233e-12, 5.04555942e-07, 4.58987720e-13, 9.99846339e-01,
        1.21066792e-08, 4.88949809e-07],
       [3.38340307e-08, 4.00497328e-07, 9.99998689e-01, 7.95214248e-07,
        4.45750400e-16, 1.23090151e-08, 4.26582751e-08, 1.21889220e-13,
        6.61571988e-08, 2.68813596e-12],
       [1.02244606e-07, 9.99130547e-01, 4.59575240e-05, 1.72651846e-06,
        3.47284586e-05, 1.14181294e-05, 1.07853002e-05, 7.33653957e-04,
        3.02470544e-05, 8.58652697e-07],
       [9.99970078e-01, 9.96469296e-11, 7.01584099e-07, 3.28843868e-08,
        8.77174156e-09, 2.99583292e-07, 2.41563412e-05, 1.06112077e-06,
        9.09705289e-09, 3.53195537e-06],
       [1.94431300e-06, 4.08671408e-09, 7.60603314e-07, 9.49914103e-09,
        9.99241590e-01, 1.63003023e-07, 6.86637634e-07, 2.81614975e-05,
        2.27033524e-06, 7.24473794e-04]], dtype=float32)>

**Copyright 2019 The TensorFlow Authors.**

In [13]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.